# CS4248 Project
(DON'T CLICK RUN ALL! Lemmatization takes tooo long and pls don't run it again)

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# %cd "/content/drive/My Drive/CS4248 Project"
# !cd "/content/drive/My Drive/CS4248 Project"

%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
X_train = train['text']
y_train = train['class']

In [8]:
train.head()

,class,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [9]:
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])
X_test = test['text']
y_test = test['class']

In [10]:
test.head()

,class,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


## Baseline Models

In [11]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,1), max_features = 10000)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = tfidf(X_train, X_test)

In [71]:
# Logistic Regression - baseline

def LR(X_train_tfidf, y_train, X_test_tfidf, y_test):

  LR_classifier = LogisticRegression(random_state = 42, max_iter=1000).fit(X_train_tfidf, y_train)

  y_pred_lr = LR_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_lr))

  return y_pred_lr

y_pred_lr = LR(X_train_tfidf, y_train, X_test_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.85      0.79      0.82       750
           2       0.81      0.37      0.51       750
           3       0.56      0.83      0.67       750
           4       0.81      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.76      0.73      0.71      3000
weighted avg       0.76      0.73      0.71      3000


In [72]:
# NB - baseline

def NB(X_train_tfidf, y_train, X_test_tfidf, y_test):

  nb_classifier = ComplementNB().fit(X_train_tfidf, y_train)

  y_pred_nb = nb_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_nb))

  return y_pred_nb

y_pred_nb = NB(X_train_tfidf, y_train, X_test_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.62      0.61      0.61       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.68      0.67      0.66      3000
weighted avg       0.68      0.67      0.66      3000


## Preprocessing

### Lemmatization with POS Tagging
(DON'T RUN AGAIN! It takes tooooo long)

In [14]:
# # WORDNET LEMMATIZER (with appropriate pos tags)
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
# 
# lemmatizer = WordNetLemmatizer()
# 
# # Define function to lemmatize each word with its POS tag
# 
# # POS_TAGGER_FUNCTION : TYPE 1
# def pos_tagger(nltk_tag):
#     if nltk_tag.startswith('J'):
#         return wordnet.ADJ
#     elif nltk_tag.startswith('V'):
#         return wordnet.VERB
#     elif nltk_tag.startswith('N'):
#         return wordnet.NOUN
#     elif nltk_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return None
# 
# def lemmatize_with_pos(tokens):
#   # tokenize the sentence and find the POS tag for each token
#   pos_tagged = nltk.pos_tag(tokens)
# 
#   # print(pos_tagged)
# 
#   wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
#   # print(wordnet_tagged)
# 
#   lemmatized_sentence = []
#   for word, tag in wordnet_tagged:
#       if tag is None:
#           # if there is no available tag, append the token as is
#           lemmatized_sentence.append(word)
#       else:
#           # else use the tag to lemmatize the token
#           lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
#   # lemmatized_sentence = " ".join(lemmatized_sentence)
# 
#   # print(lemmatized_sentence)
#   return lemmatized_sentence

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
# # Start preprocessing
# 
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from tqdm import tqdm
from spellchecker import SpellChecker
# 
# punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
# 
# spell = SpellChecker()
# 
# def preprocessing(data):
#   data_clean = []
#   for sentence in tqdm(data):
# 
#     # Tokenization
#     tokens = word_tokenize(sentence)
# 
#     # Remove punctuation and number
#     tokens = [w for w in tokens if (not w in punc) and (not w.isdigit())]
# 
#     # Spell check
#     # Taking too long time for each sentence, not practical to be used
#     # tokens = [spell.correction(w) for w in tokens]
#     # tokens = [w for w in tokens if w is not None and len(w) > 0]
# 
#     # Lemmatization based on tagging
#     tokens = lemmatize_with_pos(tokens)
# 
#     data_clean.append((' ').join(tokens))
#   return data_clean
# 
# X_train_clean = preprocessing(X_train)
# X_test_clean = preprocessing(X_test)

[nltk_data] Downloading package punkt to /Users/stella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stella/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [79]:
# import csv
# import pandas as pd
# 
# def write_to_csv(X_train_clean, X_test_clean):
#     with open('raw_data/X_train_clean.csv', 'w', newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         for doc in X_train_clean:
#             writer.writerow([doc])
# 
#     with open('raw_data/X_test_clean.csv', 'w', newline='', encoding='utf-8') as f:
#         writer = csv.writer(f)
#         for doc in X_test_clean:
#             writer.writerow([doc])
# 
# # Uncomment if you want to save X_train_clean and X_test_clean
# # write_to_csv(X_train_clean, X_test_clean)

In [12]:
def read_from_file():
    a_df = pd.read_csv('raw_data/X_train_clean.csv', header=None, dtype=str)
    a = a_df[0].values
    
    b_df = pd.read_csv('raw_data/X_test_clean.csv', header=None, dtype=str)
    b = b_df[0].values
    return a, b

# Uncomment if you want to read from file
a, b = read_from_file()
X_train_clean, X_test_clean = a, b

In [13]:
X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

_ = LR(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

In [82]:
_ = NB(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.65      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


### Removing stopwords, duplicates, and contradicting labels

In [83]:
def lowercase(text): #trivial casefolding
    return text.lower()

#Obtain set of stopwords from nltk
stop_words = set(stopwords.words('english'))

def remove_stopwords(data):
    data_removed_stopwords = []
    for sentence in tqdm(data):
        #tokenize sentence into words
        words = word_tokenize(sentence)
        
        #remove word if found in set of stopwords
        words_wo_stopwords = [word for word in words if word.lower() not in stop_words]
        
        data_removed_stopwords.append(' '.join(words_wo_stopwords))
    return data_removed_stopwords

def remove_dupes(data):
    #use pandas to drop duplicates, only removes those with equal values in both class and text columns
    return data.drop_duplicates()

def remove_contradictions(data):
    #group by text and filter groups with more than one unique label(contradicting labels)
    contradictions = data.groupby('text').filter(lambda x: x['class'].nunique() > 1)
    
    #remove all texts with contradicting labels since we do not know which is the correct label
    return data[~data['text'].isin(contradictions['text'])]

def preprocess2(data):
    processed_data = remove_dupes(data)
    processed_data = remove_contradictions(processed_data)
    processed_data['text'] = remove_stopwords(processed_data['text'])
    
    return processed_data

train_clean2 = preprocess2(train)    
test_clean2 = preprocess2(test)    

100%|██████████| 2990/2990 [00:04<00:00, 606.07it/s]


In [84]:
X_train_clean2 = train_clean2['text']
y_train_clean2 = train_clean2['class']
X_test_clean2 = test_clean2['text']
y_test_clean2 = test_clean2['class']

X_train_clean2_tfidf, X_test_clean2_tfidf = tfidf(X_train_clean2, X_test_clean2)

In [85]:
_ = LR(X_train_clean2_tfidf, y_train_clean2, X_test_clean2_tfidf, y_test_clean2)

              precision    recall  f1-score   support

           1       0.85      0.74      0.79       745
           2       0.79      0.42      0.55       750
           3       0.57      0.83      0.67       747
           4       0.81      0.91      0.86       748

    accuracy                           0.73      2990
   macro avg       0.75      0.73      0.72      2990
weighted avg       0.75      0.73      0.72      2990


In [86]:
_ = NB(X_train_clean2_tfidf, y_train_clean2, X_test_clean2_tfidf, y_test_clean2)

              precision    recall  f1-score   support

           1       0.60      0.60      0.60       745
           2       0.61      0.45      0.52       750
           3       0.61      0.92      0.73       747
           4       0.87      0.67      0.76       748

    accuracy                           0.66      2990
   macro avg       0.67      0.66      0.65      2990
weighted avg       0.67      0.66      0.65      2990


In [18]:
def load_glove_model(glove_file_path):
    glove_model = {}
    with open(glove_file_path, 'r', encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            # embedding = np.array([float(val) for val in split_line[1:]])
            try:
                coefs = np.asarray(split_line[1:], dtype='float32')
            except ValueError:
                pass
            glove_model[word] = coefs
    return glove_model

glove_model = load_glove_model("glove/glove.6B.300d.txt")

In [19]:
def text_to_features(tokens, glove_model):
    vectors = [glove_model[token] for token in tokens if token in glove_model]
    if vectors:
        return np.mean(vectors, axis=0) # Return the mean of the vectors
    else:
        return np.zeros(300)  # Return a zero vector if no words are found

# Convert X_train_clean to a pandas Series
X_train_clean_series = pd.Series(X_train_clean)
# Convert text to features using a predefined function (e.g., using GloVe embeddings)
glove_embedding_features = X_train_clean_series.apply(lambda x: pd.Series(text_to_features(x, glove_model)))
glove_embedding_features.columns = [f'Feature_{i}' for i in range(300)]

# Use the same function to convert X_test_clean to a pandas Series
X_test_clean_series = pd.Series(X_test_clean)
glove_embedding_features_test = X_test_clean_series.apply(lambda x: pd.Series(text_to_features(x, glove_model)))
glove_embedding_features_test.columns = [f'Feature_{i}' for i in range(300)]

## SVM

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def SVM(X_train, y_train, X_test, y_test):
    le = LabelEncoder()
    y_train_en = le.fit_transform(y_train)
    y_test_en = le.transform(y_test)
    model = svm.SVC(kernel='linear', class_weight='balanced', probability=True, verbose=True) # Best Model chosen by Grid Search
    model.fit(X_train, y_train_en)
    predictions_y = model.predict(X_test)
    print("SVM Accuracy Score -> ", accuracy_score(predictions_y, y_test_en) * 100)
    print(classification_report(y_test_en, predictions_y))
    cm = confusion_matrix(y_test_en, predictions_y)
    print("Confusion Matrix:")
    print(cm)

### Approach 1: tfidf 

In [17]:
_ = SVM(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

[LibSVM]....*...*
optimization finished, #iter = 7139
obj = -789.401718, rho = 0.700044
nSV = 2220, nBSV = 397
Total nSV = 2220
....*...*
optimization finished, #iter = 7298
obj = -795.939370, rho = 0.746327
nSV = 2270, nBSV = 409
Total nSV = 2270
....*...*
optimization finished, #iter = 7141
obj = -787.459236, rho = 0.742474
nSV = 2204, nBSV = 381
Total nSV = 2204
....*..*
optimization finished, #iter = 6952
obj = -790.666170, rho = 0.684879
nSV = 2243, nBSV = 421
Total nSV = 2243
....*...*
optimization finished, #iter = 7207
obj = -806.421280, rho = 0.734366
nSV = 2247, nBSV = 406
Total nSV = 2247
.....*...*
optimization finished, #iter = 8148
obj = -912.494670, rho = -0.746157
nSV = 2491, nBSV = 853
......*...*
optimization finished, #iter = 9700
obj = -1260.480464, rho = -0.322423
nSV = 3845, nBSV = 1587
Total nSV = 3845
......*...*
optimization finished, #iter = 9950
obj = -1239.615416, rho = -0.348236
nSV = 3873, nBSV = 1587
Total nSV = 3873
......*....*
optimization finished, #i

### Approach 2: word embedding

In [20]:
_ = SVM(glove_embedding_features, y_train, glove_embedding_features_test, y_test)

[LibSVM]........
*
optimization finished, #iter = 8601
obj = -7726.134365, rho = 8.175568
nSV = 7424, nBSV = 6370
Total nSV = 7424
........
*
optimization finished, #iter = 8660
obj = -7725.702620, rho = 5.582771
nSV = 7428, nBSV = 6381
Total nSV = 7428
........
*.
*
optimization finished, #iter = 9338
obj = -7705.216372, rho = 6.200868
nSV = 7404, nBSV = 6149
Total nSV = 7404
........
*.
*
optimization finished, #iter = 8924
obj = -7755.065654, rho = 6.579171
nSV = 7453, nBSV = 6249
Total nSV = 7453
........
*.
*
optimization finished, #iter = 9167
obj = -7631.088532, rho = 5.936715
nSV = 7352, nBSV = 6234
Total nSV = 7352
..........
*
optimization finished, #iter = 10888
obj = -9401.277879, rho = -5.954387
nSV = 8970, nBSV = 5482
............
*.
*.
*
optimization finished, #iter = 13579
obj = -7465.246804, rho = 9.180654
nSV = 11019, nBSV = 6933
Total nSV = 11019
............
*.
*.
*.
*
optimization finished, #iter = 14214
obj = -7436.760228, rho = 8.695698
nSV = 10985, nBSV = 6446
T

### Approach 3: tfidf + word embedding

In [21]:
from scipy.sparse import hstack

def add_glove_to_tfidf(tfidf_matrix, glove_embedding_matrix):
    enhanced_tfidf_matrix = hstack([tfidf_matrix, glove_embedding_matrix])
    return enhanced_tfidf_matrix

X_train_tfidf_glove = add_glove_to_tfidf(X_train_clean_tfidf, glove_embedding_features)
X_test_tfidf_glove = add_glove_to_tfidf(X_test_clean_tfidf, glove_embedding_features_test)
_ = SVM(X_train_tfidf_glove, y_train, X_test_tfidf_glove, y_test)

[LibSVM]....*..*
optimization finished, #iter = 6890
obj = -673.638767, rho = 3.084995
nSV = 2027, nBSV = 241
Total nSV = 2027
....*..*
optimization finished, #iter = 6974
obj = -663.625045, rho = 3.053985
nSV = 2014, nBSV = 271
Total nSV = 2014
....*..*
optimization finished, #iter = 6883
obj = -663.761947, rho = 3.563594
nSV = 2027, nBSV = 231
Total nSV = 2027
....*...*
optimization finished, #iter = 7088
obj = -666.107323, rho = 3.448630
nSV = 2029, nBSV = 270
Total nSV = 2029
....*..*
optimization finished, #iter = 6742
obj = -666.680352, rho = 3.128750
nSV = 1998, nBSV = 252
Total nSV = 1998
.....*...*
optimization finished, #iter = 8094
obj = -764.558916, rho = -3.525002
nSV = 2231, nBSV = 698
......*...*
optimization finished, #iter = 9901
obj = -936.454228, rho = 3.170725
nSV = 3169, nBSV = 1143
Total nSV = 3169
......*...*
optimization finished, #iter = 9630
obj = -946.170038, rho = 3.103946
nSV = 3214, nBSV = 1165
Total nSV = 3214
......*....*
optimization finished, #iter = 1

### Additional Features

- Sentence Length 
- Count Booster Words


In [22]:
# Count sentence length
# Count of booster words (with preprocessing)

import pandas as pd
import numpy as np
import re
import nltk
from scipy.sparse import hstack
import time

# Uncomment if you have not downloaded the 'punkt' and 'averaged_perceptron_tagger' packages
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')

def add_custom_feature_to_tfidf(data, tfidf_matrix, feature_extraction_func):
    custom_feature = data.apply(feature_extraction_func)
    custom_feature_sparse = np.array(custom_feature).reshape(-1, 1)
    enhanced_tfidf_matrix = hstack([tfidf_matrix, custom_feature_sparse])
    return enhanced_tfidf_matrix

# Feature extraction functions
def count_sentence_length(text):
    return len(nltk.sent_tokenize(text))

def count_quotations(text):
    return text.count('"')

def count_numbers(text):
    return len(re.findall(r'\d+', text))

def count_proper_nouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    proper_nouns = [word for word, pos in tagged if pos == 'NNP']
    return len(proper_nouns)

def count_conjunctions(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    conjunctions = [word for word, pos in tagged if pos == 'CC']
    return len(conjunctions)

def count_superlatives(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    superlatives = [word for word, pos in tagged if pos == 'JJS']
    return len(superlatives)

def count_1st_pronouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    first_pronouns = [word for word, pos in tagged if pos == 'PRP' and word.lower() in ['i', 'me', 'my', 'mine']]
    return len(first_pronouns)

def count_3rd_pronouns(text):
    tagged = nltk.pos_tag(nltk.word_tokenize(text))
    third_pronouns = [word for word, pos in tagged if pos == 'PRP' and word.lower() in 
                      ['he', 'him', 'his', 'she', 'her', 'hers', 'it', 'its', 'they', 'them', 'their', 'theirs']]
    return len(third_pronouns)

def count_booster_words(text):
    booster_words = """absolutely amazingly awfully completely considerably definitely entirely exceedingly
                    extremely highly incredibly indeed quite really remarkably so terribly totally truly utterly very"""
    booster_words = booster_words.split(" ")
    return len([word for word in text.split(" ") if word in booster_words])

# Evaluate the performance of the model with the custom features
# For the purpose of comparison, we can just use the logistic regression model across all the custom features
features_methods_dictionary =  {
    "Sentence Length": count_sentence_length, #ok
    # "Count Quotations": count_quotations,
    # "Count Numbers": count_numbers,
    # "Count Proper Nouns": count_proper_nouns,
    # "Count Conjunctions": count_conjunctions,
    # "Count Superlatives": count_superlatives,
    # "Count 1st Pronouns": count_1st_pronouns,
    # "Count 3rd Pronouns": count_3rd_pronouns,
    "Count Booster Words": count_booster_words # ok
}

training_datasets = {
    # "Not Processed": [X_train, X_train_tfidf, X_test, X_test_tfidf],
    "Processed": [X_train_clean, X_train_clean_tfidf, X_test_clean, X_test_clean_tfidf]
}
for dataset_name , datasets in training_datasets.items():
    X_train = datasets[0]
    X_train_tfidf = datasets[1]
    X_test = datasets[2]
    X_test_tfidf = datasets[3]
    if not (isinstance(X_train, pd.Series) or isinstance(X_train, pd.DataFrame)):
        X_train = pd.Series(X_train)
    if not (isinstance(X_test, pd.Series) or isinstance(X_test, pd.DataFrame)):
        X_test = pd.Series(X_test)
    for feature_name, feature_matrix in features_methods_dictionary.items():
        time_start = time.time()
        if dataset_name == "Not Processed":
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        else:
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        _ = SVM(X_train_enhanced, y_train, X_test_enhanced, y_test)
        time_end = time.time()
        print(f"Time taken for {feature_name} on {dataset_name} dataset: {time_end - time_start} seconds")

[nltk_data] Downloading package punkt to /Users/stella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/stella/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[LibSVM]....*....*.*
optimization finished, #iter = 9456
obj = -790.273763, rho = 1.006198
nSV = 2245, nBSV = 454
Total nSV = 2245
......*..*
optimization finished, #iter = 8024
obj = -799.318501, rho = 0.965214
nSV = 2231, nBSV = 354
Total nSV = 2231
......*..*
optimization finished, #iter = 8013
obj = -767.939409, rho = 0.994926
nSV = 2200, nBSV = 391
Total nSV = 2200
.....*..*
optimization finished, #iter = 7940
obj = -787.390706, rho = 1.050144
nSV = 2237, nBSV = 329
Total nSV = 2237
......*..*
optimization finished, #iter = 8076
obj = -785.092674, rho = 1.008163
nSV = 2229, nBSV = 321
Total nSV = 2229
......*....*.*
optimization finished, #iter = 10565
obj = -903.863326, rho = -1.027343
nSV = 2470, nBSV = 832
.......*...*
optimization finished, #iter = 10564
obj = -1243.698809, rho = -0.130755
nSV = 3880, nBSV = 1423
Total nSV = 3880
.......*...*
optimization finished, #iter = 10925
obj = -1254.426610, rho = -0.144228
nSV = 3883, nBSV = 1459
Total nSV = 3883
.......*...*
optimizat

## Tune Hyperparameter 

In [26]:
def SVM2(X_train, y_train, X_test, y_test):
    le = LabelEncoder()
    y_train_en = le.fit_transform(y_train)
    y_test_en = le.transform(y_test)
    model = svm.SVC(kernel='rbf', gamma="auto", class_weight='balanced', probability=True, verbose=True) # Best Model chosen by Grid Search
    model.fit(X_train, y_train_en)
    predictions_y = model.predict(X_test)
    print("SVM Accuracy Score -> ", accuracy_score(predictions_y, y_test_en) * 100)
    print(classification_report(y_test_en, predictions_y))
    cm = confusion_matrix(y_test_en, predictions_y)
    print("Confusion Matrix:")
    print(cm)

In [27]:

features_methods_dictionary =  {
    # "Sentence Length": count_sentence_length, #ok
    # "Count Quotations": count_quotations,
    # "Count Numbers": count_numbers,
    # "Count Proper Nouns": count_proper_nouns,
    # "Count Conjunctions": count_conjunctions,
    # "Count Superlatives": count_superlatives,
    # "Count 1st Pronouns": count_1st_pronouns,
    # "Count 3rd Pronouns": count_3rd_pronouns,
    "Count Booster Words": count_booster_words # ok
}

training_datasets = {
    # "Not Processed": [X_train, X_train_tfidf, X_test, X_test_tfidf],
    "Processed": [X_train_clean, X_train_clean_tfidf, X_test_clean, X_test_clean_tfidf]
}
for dataset_name , datasets in training_datasets.items():
    X_train = datasets[0]
    X_train_tfidf = datasets[1]
    X_test = datasets[2]
    X_test_tfidf = datasets[3]
    if not (isinstance(X_train, pd.Series) or isinstance(X_train, pd.DataFrame)):
        X_train = pd.Series(X_train)
    if not (isinstance(X_test, pd.Series) or isinstance(X_test, pd.DataFrame)):
        X_test = pd.Series(X_test)
    for feature_name, feature_matrix in features_methods_dictionary.items():
        time_start = time.time()
        if dataset_name == "Not Processed":
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        else:
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        _ = SVM2(X_train_enhanced, y_train, X_test_enhanced, y_test)
        time_end = time.time()
        print(f"Time taken for {feature_name} on {dataset_name} dataset: {time_end - time_start} seconds")

[LibSVM]...............
**.
*
optimization finished, #iter = 15333
obj = -16876.152114, rho = -0.311509
nSV = 15163, nBSV = 11124
Total nSV = 15163
...............
**.
*
optimization finished, #iter = 15365
obj = -16886.100506, rho = -0.334201
nSV = 15177, nBSV = 11148
Total nSV = 15177
...............
*
optimization finished, #iter = 15326
obj = -16820.405920, rho = -0.359268
nSV = 15117, nBSV = 11086
Total nSV = 15117
...............
**.
*
optimization finished, #iter = 15294
obj = -16800.403760, rho = -0.343364
nSV = 15104, nBSV = 11063
Total nSV = 15104
...............
**.
*
optimization finished, #iter = 15343
obj = -16852.952843, rho = -0.285775
nSV = 15146, nBSV = 11121
Total nSV = 15146
..................
*.
*.
*
optimization finished, #iter = 19423
obj = -20858.305894, rho = 0.405365
nSV = 18663, nBSV = 12277
......................
*.
*.
*
optimization finished, #iter = 23688
obj = -16293.156156, rho = -0.444539
nSV = 21692, nBSV = 11761
Total nSV = 21692
.....................

/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
def SVM3(X_train, y_train, X_test, y_test):
    le = LabelEncoder()
    y_train_en = le.fit_transform(y_train)
    y_test_en = le.transform(y_test)
    model = svm.SVC(kernel='poly', class_weight='balanced', probability=True, verbose=True) # Best Model chosen by Grid Search
    model.fit(X_train, y_train_en)
    predictions_y = model.predict(X_test)
    print("SVM Accuracy Score -> ", accuracy_score(predictions_y, y_test_en) * 100)
    print(classification_report(y_test_en, predictions_y))
    cm = confusion_matrix(y_test_en, predictions_y)
    print("Confusion Matrix:")
    print(cm)

In [29]:

features_methods_dictionary =  {
    # "Sentence Length": count_sentence_length, #ok
    # "Count Quotations": count_quotations,
    # "Count Numbers": count_numbers,
    # "Count Proper Nouns": count_proper_nouns,
    # "Count Conjunctions": count_conjunctions,
    # "Count Superlatives": count_superlatives,
    # "Count 1st Pronouns": count_1st_pronouns,
    # "Count 3rd Pronouns": count_3rd_pronouns,
    "Count Booster Words": count_booster_words # ok
}

training_datasets = {
    # "Not Processed": [X_train, X_train_tfidf, X_test, X_test_tfidf],
    "Processed": [X_train_clean, X_train_clean_tfidf, X_test_clean, X_test_clean_tfidf]
}
for dataset_name , datasets in training_datasets.items():
    X_train = datasets[0]
    X_train_tfidf = datasets[1]
    X_test = datasets[2]
    X_test_tfidf = datasets[3]
    if not (isinstance(X_train, pd.Series) or isinstance(X_train, pd.DataFrame)):
        X_train = pd.Series(X_train)
    if not (isinstance(X_test, pd.Series) or isinstance(X_test, pd.DataFrame)):
        X_test = pd.Series(X_test)
    for feature_name, feature_matrix in features_methods_dictionary.items():
        time_start = time.time()
        if dataset_name == "Not Processed":
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        else:
            X_train_enhanced = add_custom_feature_to_tfidf(X_train, X_train_tfidf, feature_matrix)
            X_test_enhanced = add_custom_feature_to_tfidf(X_test, X_test_tfidf, feature_matrix)
        _ = SVM3(X_train_enhanced, y_train, X_test_enhanced, y_test)
        time_end = time.time()
        print(f"Time taken for {feature_name} on {dataset_name} dataset: {time_end - time_start} seconds")

[LibSVM]...............
*.
*
optimization finished, #iter = 16058
obj = -16835.844554, rho = -0.999347
nSV = 15073, nBSV = 11450
Total nSV = 15073
...............
*
optimization finished, #iter = 15177
obj = -16673.883936, rho = -0.999475
nSV = 14744, nBSV = 11482
Total nSV = 14744
...............
*.
*
optimization finished, #iter = 15974
obj = -16737.495439, rho = -0.999442
nSV = 14973, nBSV = 13744
Total nSV = 14973
...............
*.
*
optimization finished, #iter = 15903
obj = -16881.456089, rho = -0.999345
nSV = 15075, nBSV = 11706
Total nSV = 15075
...............
*.
*
optimization finished, #iter = 15965
obj = -16886.380313, rho = -0.999356
nSV = 15078, nBSV = 11622
Total nSV = 15078
....................
*
optimization finished, #iter = 20060
obj = -20844.967505, rho = 0.998979
nSV = 18672, nBSV = 12297
.......................
*........
*...............................................................................................................................................